In [36]:
import os
from pathlib import Path
import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from sklearn.metrics import roc_auc_score, f1_score, confusion_matrix, accuracy_score

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

DATA_DIR = Path("/content/drive/MyDrive/chest_xray")
TRAIN_DIR = DATA_DIR / "train"
TEST_DIR = DATA_DIR / "test"

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", DEVICE)
print(DATA_DIR.exists())


Device: cuda
True


In [37]:
class CXRFolder(Dataset):
    def __init__(self, root_dir, transform=None):
        self.samples = []
        self.transform = transform
        root_dir = Path(root_dir)
        for label_name in ["NORMAL", "PNEUMONIA"]:
            lab = 0 if label_name == "NORMAL" else 1
            folder = root_dir / label_name
            if not folder.exists():
                continue
            for p in folder.iterdir():
                if p.suffix.lower() in [".png", ".jpg", ".jpeg"]:
                    self.samples.append((str(p), lab))

        random.shuffle(self.samples)

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        p, y = self.samples[idx]
        img = Image.open(p).convert("RGB")
        if self.transform:
            img = self.transform(img)
        return img, torch.tensor(y, dtype=torch.float32)

print("Train images:", len(list((TRAIN_DIR/"NORMAL").glob("*"))) + len(list((TRAIN_DIR/"PNEUMONIA").glob("*"))))


Train images: 5216


In [38]:
IMG_SIZE = 224

train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8,1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])

val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406], std=[0.229,0.224,0.225]),
])


In [39]:
train_ds = CXRFolder(TRAIN_DIR, transform=train_transforms)
test_ds  = CXRFolder(TEST_DIR,  transform=val_transforms)

BATCH_SIZE = 64
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
test_loader  = DataLoader(test_ds,  batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

print("Train samples:", len(train_ds), "Test samples:", len(test_ds))


Train samples: 5216 Test samples: 624


In [40]:
import torchvision

model = torchvision.models.densenet121(pretrained=True)
num_ftrs = model.classifier.in_features
model.classifier = nn.Linear(num_ftrs, 1)
model = model.to(DEVICE)

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max', factor=0.5, patience=2)


In [41]:
def evaluate(model, loader, device):
    model.eval()
    ys, preds = [], []
    with torch.no_grad():
        for xb, yb in loader:
            xb = xb.to(device)
            yb = yb.to(device)
            out = model(xb).squeeze(1)
            probs = torch.sigmoid(out).cpu().numpy()
            ys.extend(yb.cpu().numpy().tolist())
            preds.extend(probs.tolist())
    y_true = np.array(ys)
    y_pred = np.array(preds)
    try:
        auc = roc_auc_score(y_true, y_pred)
    except ValueError:
        auc = float("nan")
    y_bin = (y_pred >= 0.5).astype(int)
    acc = accuracy_score(y_true, y_bin)
    f1 = f1_score(y_true, y_bin, zero_division=0)
    cm = confusion_matrix(y_true, y_bin)
    return {"auc":auc, "acc":acc, "f1":f1, "cm":cm}


In [ ]:
EPOCHS = 8
best_auc = 0.0
for epoch in range(1, EPOCHS+1):
    model.train()
    running_loss = 0.0
    pbar = tqdm(train_loader, desc=f"Epoch {epoch}/{EPOCHS}")
    for xb, yb in pbar:
        xb = xb.to(DEVICE)
        yb = yb.to(DEVICE)
        optimizer.zero_grad()
        out = model(xb).squeeze(1)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * xb.size(0)
        pbar.set_postfix(loss=loss.item())
    epoch_loss = running_loss / len(train_ds)
    val_metrics = evaluate(model, test_loader, DEVICE)
    print(f"Epoch {epoch} loss: {epoch_loss:.4f}  val_auc: {val_metrics['auc']:.4f}  acc: {val_metrics['acc']:.4f} f1: {val_metrics['f1']:.4f}")
    scheduler.step(val_metrics['auc'])
    if val_metrics['auc'] > best_auc:
        best_auc = val_metrics['auc']
        torch.save(model.state_dict(), "best_densenet121.pth")
        print("Saved best model. AUC:", best_auc)


Epoch 1/3: 100%|██████████| 82/82 [02:24<00:00,  1.76s/it, loss=0.0876]


Epoch 1 loss: 0.1647  val_auc: 0.9757  acc: 0.9006 f1: 0.9262
Saved best model. AUC: 0.9757396449704142


Epoch 2/3: 100%|██████████| 82/82 [01:53<00:00,  1.38s/it, loss=0.202]


Epoch 2 loss: 0.0784  val_auc: 0.9842  acc: 0.9103 f1: 0.9329
Saved best model. AUC: 0.9841880341880341


Epoch 3/3: 100%|██████████| 82/82 [01:54<00:00,  1.40s/it, loss=0.0195]


Epoch 3 loss: 0.0514  val_auc: 0.9843  acc: 0.9487 f1: 0.9597
Saved best model. AUC: 0.9842537804076266


In [ ]:

import numpy as np
m = evaluate(model, test_loader, DEVICE)
print("AUC:", m["auc"])
print("Accuracy:", m["acc"])
print("F1:", m["f1"])
print("Confusion matrix:\n", m["cm"])


In [ ]:
model.eval()
dummy = torch.randn(1, 3, IMG_SIZE, IMG_SIZE).to(DEVICE)
onnx_path = "/content/drive/MyDrive/pneumonia_model.onnx"

torch.onnx.export(
    model,
    dummy,
    onnx_path,
    input_names=["input"],
    output_names=["output"],
    dynamic_axes={"input": {0: "batch"}},
    opset_version=17
)

print(f"✅ ONNX model saved to: {onnx_path}")